<a href="https://colab.research.google.com/github/ramanakurva164/genai/blob/main/RAG_S2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the required libraries:
# transformers: for using pre-trained models from Hugging Face
# faiss-cpu: for efficient similarity search
# pymupdf: for extracting text from PDF files
# sentence-transformers: for creating sentence embeddings
!pip install transformers faiss-cpu pymupdf sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [17]:
# Import the files module from google.colab
from google.colab import files
# Upload a file from your local machine
uploaded=files.upload()
# Get the path of the uploaded PDF file
pdf_path=list(uploaded.keys())[0]

Saving Copy of Campbell Essential Biology with Physiology Eric J_ Simon, Jean L_ Dickey, Jane B_ Reece, Kelly A_ Hogan 5, 2015.pdf to Copy of Campbell Essential Biology with Physiology Eric J_ Simon, Jean L_ Dickey, Jane B_ Reece, Kelly A_ Hogan 5, 2015.pdf


In [19]:
# Import the fitz library (PyMuPDF)
import fitz
# Define a function to extract text from a PDF file
def return_text_form_pdf(pdf_path):
  # Open the PDF file
  doc=fitz.open(pdf_path)
  # Initialize an empty string to store the text
  text=""
  # Iterate through each page of the PDF
  for page in doc:
      # Get the text from the current page and append it to the text string
      text+=page.get_text()
  # Return the extracted text
  return text
# Call the function to get the text from the uploaded PDF
document_text=return_text_form_pdf(pdf_path)

chunking


In [20]:
# Import the textwrap library for text formatting
import textwrap
# Define a function to chunk text into smaller pieces
def chunk_text(text,chunk_size=300):
  # Use textwrap.wrap to split the text into chunks of a specified size
  return textwrap.wrap(text,width=chunk_size)

# Chunk the document text into smaller pieces
chunks=chunk_text(document_text)
# Print the number of chunks created
print(f"no.of.chunks{len(chunks)}")
# Print a sample chunk
print("sample_chunk",chunks[0])

no.of.chunks8764
sample_chunk Global  edition  Eric J. Simon • Jean L. Dickey • Kelly A. Hogan • Jane B. Reece Campbell   Essential Biology with Physiology  fifTH edition                           This page is intentionally left blank.           1        Introduction: Biology Today  36         UNIT 1   Cells  55          2


In [21]:
# Import the SentenceTransformer library for creating sentence embeddings
from sentence_transformers import SentenceTransformer
# Import the faiss library for efficient similarity search
import faiss
# Import the numpy library for numerical operations
import numpy as np

# Load a pre-trained sentence transformer model
embed_model=SentenceTransformer("all-MiniLM-L6-v2")

# Encode the text chunks into numerical embeddings
embeddings=embed_model.encode(chunks)

# Get the dimension of the embeddings
dim=embeddings[0].shape[0]
# Create a faiss index for efficient similarity search
index=faiss.IndexFlatL2(dim)
# Add the embeddings to the faiss index
index.add(np.array(embeddings))

Retrivel + generation

In [22]:
# Import the pipeline function from the transformers library
from transformers import pipeline

# Create a text2text-generation pipeline using the "google/flan-t5-small" model
generator=pipeline("text2text-generation",model="google/flan-t5-small")

# Define a function to retrieve relevant chunks and generate an answer
def retrive_and_answer(query,top_k=1):
    # Encode the query into a numerical embedding
    query_embedding=embed_model.encode([query])
    # Search the faiss index for the most similar chunks to the query
    _,indices=index.search(np.array(query_embedding),top_k)
    # Retrieve the most relevant chunks
    retrieved_chunks=[chunks[i] for i in indices[0]]
    # Combine the retrieved chunks into a single context string
    context=" ".join(retrieved_chunks)
    # Generate an answer using the generator pipeline with the query and context
    result=generator(f"question: {query} context: {context}")
    # Return the generated answer
    return result[0]['generated_text']

Device set to use cpu


In [33]:
# Define the question to be answered
Question="what is eukariotik cell?"
# Print the question to the console
print(Question)
# Call the retrive_and_answer function with the question and print the result
print(retrive_and_answer(Question))

what is eukariotik cell?
epi- dermis
